In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-sample.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-labels.pkl

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['MALAYA_USE_HUGGINGFACE'] = 'true'

In [3]:
import malaya_speech
import json
import pickle
from tqdm import tqdm

In [4]:
with open('/home/husein/youtube/voxceleb2-test-sample.json') as fopen:
    sample_files = json.load(fopen)

In [5]:
with open('/home/husein/youtube/voxceleb2-test-labels.pkl', 'rb') as fopen:
    labels = pickle.load(fopen)

In [6]:
model = malaya_speech.speaker_vector.deep_model(model = 'speakernet')

In [8]:
unique_files = []
for l in labels:
    unique_files.extend(l[1:])
    
unique_files = list(set(unique_files))  

In [9]:
unique_files[0]

'/home/husein/youtube/voxceleb-wav/aac-id01333-2aUbLFU7mV8-00032.wav'

In [10]:
vectors = {}
for f in tqdm(unique_files):
    y, _ = malaya_speech.load(f)
    v = model.vectorize([y])[0]
    vectors[f] = v

100%|██████████| 36237/36237 [48:22<00:00, 12.48it/s]  


In [11]:
import numpy as np

scores, ls = [], []

for i in tqdm(range(len(labels))):
    ls.append(labels[i][0])
    scores.append(np.sum(vectors[labels[i][1]] * vectors[labels[i][2]]))

100%|██████████| 5900000/5900000 [01:43<00:00, 56964.45it/s]


In [12]:
len(scores)

5900000

In [13]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [14]:
calculate_eer(ls, scores)

(0.30002852459016394, array(1178.26518875))